In [ ]:
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

import socceraction.spadl.config as spadlconfig
from socceraction.spadl.schema import SPADLSchema
from pandera.typing import DataFrame, Series
import numpy.typing as npt
from sklearn.exceptions import NotFittedError
from typing import Callable, List, Optional, Tuple

In [ ]:
xTModell = xthreat.load_model(
    "/home/morten/Develop/packing-report/xT-impact/models/xT_test"
)

In [ ]:
choosen_match_id = 1557933

In [ ]:
ws = sd.WhoScored(
    leagues="GER-Bundesliga2",
    seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=True,
)

loader = ws.read_events(match_id=choosen_match_id, output_fmt="loader")

In [ ]:
df_teams = loader.teams(game_id=choosen_match_id)
df_events = loader.events(game_id=choosen_match_id)
df_players = loader.players(game_id=choosen_match_id)
display(df_events.head())

In [ ]:
# actions = {}
# actions[df_events['game_id'][0]] = spadl.opta.convert_to_actions(df_events, df_teams['team_id'][0])
# # 2. Convert direction of play
# actions_ltr = pd.concat([
#   spadl.play_left_to_right(actions[df_events['game_id'][0]], df_teams['team_id'][0])
# ])
# # succs_events = xthreat.get_successful_move_actions(actions_ltr)
# # succs_events.head()

In [ ]:
df_actions = (
    spadl.opta.convert_to_actions(df_events, df_teams["team_id"][0])
    .merge(spadl.actiontypes_df())
    .merge(df_players[["player_name", "player_id"]])
)

In [ ]:
possible_actions = ["tackle", "interception", "clearance"]  # "keeper_save"
df_all_defense = df_actions[df_actions["type_name"].isin(possible_actions)]
df_all_defense = df_all_defense[df_all_defense["result_id"] == 1]

In [ ]:
df_all_defense

In [ ]:
ratings = xTModell.rate_defensive(df_all_defense.reset_index())
ratings

In [ ]:
ws = sd.WhoScored(
    leagues="GER-Bundesliga2",
    seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=True,
)

loader = ws.read_events(output_fmt="loader")

In [ ]:
df_games = loader.games(competition_id="GER-Bundesliga2", season_id="2122")

In [ ]:
ce = sd.ClubElo(
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
)

In [ ]:
# succ_actions_dict
def succ_actions(game_id, df_events, df_games):
    actions = {}
    actions[game_id] = spadl.opta.convert_to_actions(
        df_events, df_games[df_games["game_id"] == game_id].home_team_id.values[0]
    )
    # 2. Convert direction of play
    actions_ltr = pd.concat(
        [
            spadl.play_left_to_right(
                actions[game_id],
                df_games[df_games["game_id"] == game_id].home_team_id.values[0],
            )
        ]
    )
    return actions_ltr


def fill_ids(succs_events, df_players):
    for y in succs_events["player_id"]:
        if y not in id_to_name:
            player_name = df_players[df_players["player_id"] == y][
                "player_name"
            ].values[0]
            id_to_name[y] = df_players[df_players["player_id"] == y][
                "player_name"
            ].values[0]
            id_to_xt_impact[y] = 0
            id_to_xt_impact_elo_pressing[y] = 0
            id_to_xt_impact_elo_normal[y] = 0


def reset_player_xt_values(succs_events):
    player_values = {}
    for indx, y in enumerate(succs_events["player_id"]):
        player_values[y] = 0
    return player_values


def get_club_elo_factorial(date, team, league):
    print("lol")

In [ ]:
ce = sd.ClubElo(
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/clubElo"),
)

In [ ]:
spadl.actiontypes_df()

In [ ]:
(
    id_to_name,
    id_to_xt_impact,
    id_to_xt_impact_elo_pressing,
    id_to_xt_impact_elo_normal,
) = ({}, {}, {}, {})
possible_actions = ["keeper_save", "keeper_claim", "keeper_punch"]  # , "keeper_pick_up"
# possible_actions = ["tackle", "interception", "clearance"]
for idx, game_id in enumerate(tqdm(df_games["game_id"])):
    df_teams = loader.teams(game_id=game_id)
    df_players = loader.players(game_id=game_id)
    df_events = loader.events(game_id=game_id)
    df_actions = (
        spadl.opta.convert_to_actions(
            df_events, df_games[df_games["game_id"] == game_id].home_team_id.values[0]
        )
        .merge(spadl.actiontypes_df())
        .merge(df_players[["player_name", "player_id"]])
    )
    df_actions_ltr = spadl.play_left_to_right(
        df_actions, df_games[df_games["game_id"] == game_id].home_team_id.values[0]
    )
    df_actions_rtl = spadl.play_left_to_right(
        df_actions, df_games[df_games["game_id"] == game_id].away_team_id.values[0]
    )
    df_all_defense_pressing = df_actions_ltr[
        df_actions_ltr["type_name"].isin(possible_actions)
    ]
    df_all_defense_pressing = df_all_defense_pressing[
        df_all_defense_pressing["result_id"] == 1
    ]
    df_all_defense_normal = df_actions_rtl[
        df_actions_rtl["type_name"].isin(possible_actions)
    ]
    df_all_defense_normal = df_all_defense_normal[
        df_all_defense_normal["result_id"] == 1
    ]
    # actions_ltr = succ_actions(game_id, df_events, df_games)
    # succs_events = xthreat.get_successful_move_actions(actions_ltr)
    xt_ratings_pressing = xTModell.rate_defensive(df_all_defense_pressing.reset_index())
    xt_ratings_normal = xTModell.rate_defensive(df_all_defense_normal.reset_index())
    ######################################
    fill_ids(df_all_defense_pressing, df_players)
    fill_ids(df_all_defense_normal, df_players)
    player_values_pressing = reset_player_xt_values(df_all_defense_pressing)
    player_values_normal = reset_player_xt_values(df_all_defense_normal)
    for indx, y in enumerate(df_all_defense_pressing["player_id"]):
        player_values_pressing[y] += xt_ratings_pressing[indx]
    for x in player_values_pressing.keys():
        if player_values_pressing[x] < 0:
            player_values_pressing[x] = 0
    for indx, y in enumerate(df_all_defense_normal["player_id"]):
        player_values_normal[y] += xt_ratings_normal[indx]
    for x in player_values_normal.keys():
        if player_values_normal[x] < 0:
            player_values_normal[x] = 0
    team_values_pressing = {}
    team_values_normal = {}
    df_teams = loader.teams(game_id=game_id)
    ## reset team values
    for x in df_teams["team_id"]:
        team_values_pressing[x] = 0
        team_values_normal[x] = 0
    for x in player_values_pressing.keys():
        team_values_pressing[
            df_players[df_players["player_id"] == x]["team_id"].values[0]
        ] += player_values_pressing[x]
    for x in player_values_normal.keys():
        team_values_normal[
            df_players[df_players["player_id"] == x]["team_id"].values[0]
        ] += player_values_normal[x]
    game = df_games[df_games["game_id"] == game_id]
    game_date = np.datetime_as_string(game["game_date"].values[0])[:10]
    game_comp_id = game["competition_id"].values[0]
    ce_df = ce.read_by_date(game_date)
    league_mean = np.mean(ce_df[ce_df["league"] == game_comp_id].elo)
    for x in player_values_pressing.keys():
        team_name = df_teams[
            df_teams["team_id"]
            != df_players[df_players["player_id"] == x]["team_id"].values[0]
        ]["team_name"].values[0]
        team_elo = ce_df.loc[team_name].elo
        id_to_xt_impact[x] += player_values_pressing[x] * (
            player_values_pressing[x]
            / team_values_pressing[
                df_players[df_players["player_id"] == x]["team_id"].values[0]
            ]
        )
        id_to_xt_impact_elo_pressing[x] += (
            player_values_pressing[x]
            * (
                player_values_pressing[x]
                / team_values_pressing[
                    df_players[df_players["player_id"] == x]["team_id"].values[0]
                ]
            )
        ) * (team_elo / league_mean)
    for x in player_values_normal.keys():
        team_name = df_teams[
            df_teams["team_id"]
            != df_players[df_players["player_id"] == x]["team_id"].values[0]
        ]["team_name"].values[0]
        team_elo = ce_df.loc[team_name].elo
        id_to_xt_impact[x] += player_values_normal[x] * (
            player_values_normal[x]
            / team_values_normal[
                df_players[df_players["player_id"] == x]["team_id"].values[0]
            ]
        )
        id_to_xt_impact_elo_normal[x] += (
            player_values_normal[x]
            * (
                player_values_normal[x]
                / team_values_normal[
                    df_players[df_players["player_id"] == x]["team_id"].values[0]
                ]
            )
        ) * (team_elo / league_mean)

In [ ]:
df_res = pd.DataFrame.from_dict(
    {
        "ids": [x for x in id_to_name.keys()],
        "names": [id_to_name[x] for x in id_to_name.keys()],
        "pressing_elo": [id_to_xt_impact_elo_pressing[x] for x in id_to_name.keys()],
        "normal_elo": [id_to_xt_impact_elo_normal[x] for x in id_to_name.keys()],
        "combined": [
            id_to_xt_impact_elo_pressing[x] + id_to_xt_impact_elo_normal[x]
            for x in id_to_name.keys()
        ],
    }
)
df_res.sort_values("combined", ascending=False).head(20)

In [ ]:
id_to_name, id_to_xt_impact, id_to_xt_impact_elo = {}, {}, {}
# possible_actions = ["keeper_save", "keeper_claim", "keeper_punch"] #, "keeper_pick_up"

for idx, game_id in enumerate(tqdm(df_games["game_id"])):
    df_teams = loader.teams(game_id=game_id)
    df_players = loader.players(game_id=game_id)
    df_events = loader.events(game_id=game_id)
    df_actions = (
        spadl.opta.convert_to_actions(
            df_events, df_games[df_games["game_id"] == game_id].home_team_id.values[0]
        )
        .merge(spadl.actiontypes_df())
        .merge(df_players[["player_name", "player_id"]])
    )
    df_actions_ltr = spadl.play_left_to_right(
        df_actions, df_games[df_games["game_id"] == game_id].home_team_id.values[0]
    )
    df_actions_rtl = spadl.play_left_to_right(
        df_actions, df_games[df_games["game_id"] == game_id].home_team_id.values[0]
    )
    df_all_defense_pressing = df_actions_ltr[
        df_actions_ltr["type_name"].isin(possible_actions)
    ]
    df_all_defense_pressing = df_all_defense_pressing[
        df_all_defense_pressing["result_id"] == 1
    ]
    df_all_defense_normal = df_actions_ltr[
        df_actions_rtl["type_name"].isin(possible_actions)
    ]
    df_all_defense_normal = df_all_defense_normal[
        df_all_defense_normal["result_id"] == 1
    ]
    # actions_ltr = succ_actions(game_id, df_events, df_games)
    # succs_events = xthreat.get_successful_move_actions(actions_ltr)
    xt_ratings_presssing = xTModell.rate_df_xt(df_all_defense_pressing.reset_index())
    xt_ratings_normal = xTModell.rate_df_xt(df_all_defense_normal.reset_index())
    # fill_ids(succs_events, df_players)
    for y in df_all_defense["player_id"]:
        if y not in id_to_name:
            player_name = df_players[df_players["player_id"] == y][
                "player_name"
            ].values[0]
            id_to_name[y] = df_players[df_players["player_id"] == y][
                "player_name"
            ].values[0]
            id_to_xt_impact[y] = 0
            id_to_xt_impact_elo[y] = [[], [], []]
    player_values = reset_player_xt_values(df_all_defense)
    for indx, y in enumerate(df_all_defense["player_id"]):
        player_values[y] += xt_ratings[indx]
    for x in player_values.keys():
        if player_values[x] < 0:
            player_values[x] = 0
    team_values = {}
    df_teams = loader.teams(game_id=game_id)
    ## reset team values
    for x in df_teams["team_id"]:
        team_values[x] = 0
    for x in player_values.keys():
        team_values[
            df_players[df_players["player_id"] == x]["team_id"].values[0]
        ] += player_values[x]
    game = df_games[df_games["game_id"] == game_id]
    game_date = np.datetime_as_string(game["game_date"].values[0])[:10]
    game_comp_id = game["competition_id"].values[0]
    ce_df = ce.read_by_date(game_date)
    league_mean = np.mean(ce_df[ce_df["league"] == game_comp_id].elo)
    for x in player_values.keys():
        if df_players[df_players["player_id"] == x]["is_starter"].values[0]:
            team_name = df_teams[
                df_teams["team_id"]
                != df_players[df_players["player_id"] == x]["team_id"].values[0]
            ]["team_name"].values[0]
            team_elo = ce_df.loc[team_name].elo
            id_to_xt_impact[x] += player_values[x] * (
                player_values[x]
                / team_values[
                    df_players[df_players["player_id"] == x]["team_id"].values[0]
                ]
            )
            id_to_xt_impact_elo[x][1].append(
                (
                    player_values[x]
                    * (
                        player_values[x]
                        / team_values[
                            df_players[df_players["player_id"] == x]["team_id"].values[
                                0
                            ]
                        ]
                    )
                )
                * (team_elo / league_mean)
            )
            id_to_xt_impact_elo[x][0].append(
                df_players[df_players["player_id"] == x]["minutes_played"].values[0]
            )
            id_to_xt_impact_elo[x][2].append(
                id_to_xt_impact_elo[x][1][-1] / id_to_xt_impact_elo[x][0][-1]
            )

In [ ]:
player_id = 115509
average_vec = [0.75, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.05]
weighted_av = []
for x in range(len(id_to_xt_impact_elo[player_id][2])):
    if x < 10:
        continue
    if len(weighted_av) > 0:
        tmp_arr = [weighted_av[-1]] + id_to_xt_impact_elo[player_id][2][x - 9 : x]
        weighted_av.append(np.average(tmp_arr, weights=average_vec))
    else:
        weighted_av.append(
            np.average(
                id_to_xt_impact_elo[player_id][2][x - 10 : x], weights=average_vec
            )
        )

plt.axis([0, 25, 0, 0.0005])
plt.plot(weighted_av)

In [ ]:
player_id = 77445
average_vec = [0.75, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.05]
weighted_av = []
for x in range(len(id_to_xt_impact_elo[player_id][2])):
    if x < 10:
        continue
    if len(weighted_av) > 0:
        tmp_arr = [weighted_av[-1]] + id_to_xt_impact_elo[player_id][2][x - 9 : x]
        weighted_av.append(np.average(tmp_arr, weights=average_vec))
    else:
        weighted_av.append(
            np.average(
                id_to_xt_impact_elo[player_id][2][x - 10 : x], weights=average_vec
            )
        )

plt.axis([0, 25, 0, 0.0005])
plt.plot(weighted_av)

In [ ]:
player_id = 333227
average_vec = [0.75, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.05]
weighted_av = []
for x in range(len(id_to_xt_impact_elo[player_id][2])):
    if x < 10:
        continue
    if len(weighted_av) > 0:
        tmp_arr = [weighted_av[-1]] + id_to_xt_impact_elo[player_id][2][x - 9 : x]
        weighted_av.append(np.average(tmp_arr, weights=average_vec))
    else:
        weighted_av.append(
            np.average(
                id_to_xt_impact_elo[player_id][2][x - 10 : x], weights=average_vec
            )
        )

plt.plot(weighted_av)